This script calibrates BEACO2N carbon monoxide data using QuantAQ sensors as reference. The script only applies this calibration to colocated sites, i.e. Department of Public Works, Providence Emergency Management Agency, and Providence Housing Authority. 

_TODO:_ Ensure timestamps are not becoming misaligned. [Upon manually verifying the rightmost datapoint for PHA (graphed below), the QuantAQ timestamp for that CO value did not match the graphed timestamp.]

In [ ]:
import os
from glob import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

In [39]:
# Parse the measurement and reference dataframe lists from csv files in project folder.
measurement_files = glob("./BEACO2N_measurements/*.csv")
measurement = {os.path.splitext(os.path.basename(f))[0] : pd.read_csv(f) for f in measurement_files}
reference_files = glob("./reference_measurements/*.csv")
reference = {os.path.splitext(os.path.basename(f))[0] : pd.read_csv(f) for f in reference_files}

# Clean measurement and reference data.
def clean_measurement(df: pd.DataFrame) -> pd.DataFrame :
    df = df.rename(columns={"datetime":"timestamp", "co2_raw":"co2"})
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True).dt.round("h")
    df = df.drop(columns=[col for col in ["local_timestamp", "epoch", "node_file_id"] if col in df.columns])
    wrk_aux_cols = df.filter(regex=r"_wrk_aux").columns
    df[wrk_aux_cols] *= 1000
    df.rename(columns= {col : col.replace("_wrk_aux", "") for col in wrk_aux_cols}, inplace=True)
    df = df.dropna()
    return df

measurement = {key: clean_measurement(df) for key, df in measurement.items()}

def clean_reference(df: pd.DataFrame) -> pd.DataFrame :
    df = df.drop(columns=[col for col in ["period_start", "period_end", "period_end_utc", "sn"] if col in df.columns])
    df = df.rename(columns={"period_start_utc": "timestamp", "pm25": "pm2_5"})
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    df = df.dropna()
    return df

reference = {key: clean_reference(df) for key, df in reference.items()}

# Merge measurement and reference data by site into one list of site dataframes. 
merged = {key : pd.merge(measurement[key], reference[key], on="timestamp", how="inner", suffixes=("_meas","_ref")).dropna() 
          for key in reference.keys()}

Find time intervals when RSD(co) < .10 for all reference sensors in the network:

In [42]:
co_df = []
for site, df in reference.items() :
    co_df.append(df[["timestamp","co"]].rename(columns={"co":site}))
rsd_df = co_df[0]
for df in co_df[1:]:
    rsd_df = pd.merge(rsd_df, df, on="timestamp", how="inner")

def rsd(row:pd.Series) -> float:
    vals = row[1:].values.astype(float)
    mean = np.mean(vals)
    sd = np.std(vals)
    return float(sd/mean) if mean != 0 else np.nan

rsd_df["rsd"]=rsd_df.apply(rsd, axis=1)
timestamps_rsd_lt_10pc = rsd_df[rsd_df["rsd"]<.10]["timestamp"]

# Filter datasets to include only data contained by intersection(merged, measurement, reference)

# Find the common timestamps
common_timestamps = timestamps_rsd_lt_10pc
for site in measurement:
    common_timestamps = common_timestamps[common_timestamps.isin(measurement[site]["timestamp"])]
for site in reference:
    common_timestamps = common_timestamps[common_timestamps.isin(reference[site]["timestamp"])]
for site in merged:
    common_timestamps = common_timestamps[common_timestamps.isin(merged[site]["timestamp"])]

# Filter dataframes to only include timestamps in timestamps_rsd_lt_10pc
for site in measurement.keys():
    measurement[site] = measurement[site][measurement[site]["timestamp"].isin(common_timestamps)].reset_index(drop=True).sort_values("timestamp")
for site in reference.keys():
    reference[site] = reference[site][reference[site]["timestamp"].isin(common_timestamps)].reset_index(drop=True).sort_values("timestamp")
for site in merged.keys():
    merged[site] = merged[site][merged[site]["timestamp"].isin(common_timestamps)].reset_index(drop=True).sort_values("timestamp")

# Print the first few rows of each key dataframe for inspection
print("rsd_df (first 5 rows):")
display(rsd_df.head())

print("\nmerged (first 2 rows per site):")
for k, v in merged.items():
    print(f"\nSite: {k}")
    display(v.head(2))

print("\nmeasurement (first 2 rows per site):")
for k, v in measurement.items():
    print(f"\nSite: {k}")
    display(v.head(2))

print("\nreference (first 2 rows per site):")
for k, v in reference.items():
    print(f"\nSite: {k}")
    display(v.head(2))

rsd_df (first 5 rows):


,timestamp,pha,pema,dpw,rsd
0,2024-12-17 12:00:00+00:00,754.405,744.122,701.595,0.031171
1,2024-12-17 13:00:00+00:00,740.072,754.683,716.186,0.021531
2,2024-12-17 14:00:00+00:00,734.513,762.251,717.170,0.025159
3,2024-12-17 15:00:00+00:00,753.998,777.728,725.462,0.028399
4,2024-12-17 16:00:00+00:00,744.177,788.258,746.952,0.026531



merged (first 2 rows per site):

Site: pha


,timestamp,co2,co_meas,no2_meas,no_meas,o3_meas,pm2_5_meas,pressure,rh_meas,temp_meas,...,wd,ws,pm1,pm2_5_ref,pm10,co_ref,no_ref,no2_ref,o3_ref,ws_scalar
0,2024-12-17 12:00:00+00:00,462.892583,54.672333,3.436667,-3.593667,25.721000,1.37736,1013.650033,90.182884,15.007287,...,239.755,2.118,1.017,1.968,4.642,754.405,3.398,7.128,28.949,2.335
1,2024-12-17 13:00:00+00:00,463.730000,55.103167,1.539167,-2.300833,21.973167,2.22611,1013.795214,80.274058,15.498985,...,237.424,4.287,1.651,2.289,4.248,740.072,2.726,6.928,27.273,5.033



Site: pema


...

,timestamp,n_datapoints,rh,temp,wd,ws,pm1,pm2_5,pm10,co,no,no2,o3,ws_scalar
0,2024-12-17 12:00:00+00:00,60,85.778,13.992,239.755,2.118,1.017,1.968,4.642,754.405,3.398,7.128,28.949,2.335
1,2024-12-17 13:00:00+00:00,60,82.692,13.952,237.424,4.287,1.651,2.289,4.248,740.072,2.726,6.928,27.273,5.033



Site: pema


,timestamp,n_datapoints,rh,temp,wd,ws,pm1,pm2_5,pm10,co,no,no2,o3,ws_scalar
0,2024-12-17 12:00:00+00:00,60,84.795,14.302,234.356,5.086,0.356,1.313,5.894,744.122,2.724,10.146,37.536,6.087
1,2024-12-17 13:00:00+00:00,60,81.717,14.182,227.197,5.219,0.935,1.563,4.782,754.683,2.289,9.596,34.508,6.064



Site: dpw


,timestamp,n_datapoints,rh,temp,pm1,pm2_5,pm10,co,no,no2,o3
0,2024-12-17 12:00:00+00:00,60,87.588,14.107,1.176,3.010,12.846,701.595,3.220,10.060,35.551
1,2024-12-17 13:00:00+00:00,60,84.245,14.078,1.825,2.798,7.874,716.186,2.776,10.311,32.842


Fit per-site regression models to the data where RSC(co)<0.10:

In [41]:
# Fit basic polynomial models only for the colocated BEACO2N nodes
# %pip install plotly.express
refsites = list(reference.keys())

models = {}
x_train = {}
x_test = {}
y_train = {}
y_test = {}
y_pred = {}

for i, site in enumerate(refsites):
    x_train_i, x_test_i, y_train_i, y_test_i = train_test_split(measurement[site].drop("timestamp", axis=1), reference[site]["co"], random_state=42)
    x_train[site] = x_train_i
    x_test[site] = x_test_i
    y_train[site] = y_train_i
    y_test[site] = y_test_i
    models[site] = LinearRegression()
    models[site].fit(x_train[site], y_train[site])
    y_pred[site] = models[site].predict(x_test[site])
    print(site, "\tR^2: ", models[site].score(x_test[site], y_test[site]), '\n')

for i, site in enumerate(refsites):    
    y_true = y_test[site]
    y_raw = measurement[site].sort_values(by="timestamp")["co"]
    
    fig = go.Figure()

    # Add corrected (predicted) values
    fig.add_trace(go.Scatter(
        x=y_test[site],
        y=y_pred[site],
        mode='markers',
        name='Corrected CO',
        customdata=common_timestamps.sort_values(),
        hovertemplate=
            'Reference CO: %{x}<br>' + 
            'Corrected CO:%{y}<br>' +
            'Timestamp: %{customdata}<extra></extra>'
    ))

    # Add raw (uncorrected) values
    fig.add_trace(go.Scatter(
        x=y_test[site],
        y=y_raw,
        mode='markers',
        name='Raw CO',
        customdata=common_timestamps.sort_values(),
        hovertemplate=
            'Reference CO: %{x}<br>' +
            'Raw CO: %{y}<br>' +
            'Timestamp: %{customdata}<extra></extra>'
    ))

    # Add 1:1 reference line
    min_val = min(y_test[site].min(), y_pred[site].min(), y_raw.min())
    max_val = max(y_test[site].max(), y_pred[site].max(), y_raw.max())
    fig.add_shape(
        type='line',
        x0=min_val, y0=min_val,
        x1=max_val, y1=max_val,
        line=dict(color='red', dash='dash'),
        name='1:1 Line'
    )

    fig.update_layout(
        title=f"Site: {site}",
        xaxis_title="Reference CO (mV)",
        yaxis_title="Measured CO (mV)",
        legend_title="Legend"
    )
    fig.show()

pha 	R^2:  0.9816143738945818 

pema 	R^2:  0.9844974907372382 

dpw 	R^2:  0.9826169878421204 

